In [1]:
from Networks.iResNet import iResNet2D
from Saving.saveFile import get_next_model_filename
from Trainers import split_data,Trainer
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import joblib

In [2]:
# 初始化模型
input_shape = (1,)  # 输入形状
full_connect_shape = (128, 64, 16)  # 全连接层目标形状，根据 true_output0 的形状确定
q = 2  # BB 的数量
N = 2  # 每个 BB 中 SB 的数量

output_keys = ["u"]

model = iResNet2D(input_shape, full_connect_shape, q, N, output_keys)

# 打印模型结构
print(model)

iResNet2D(
  (fc): Linear(in_features=1, out_features=131072, bias=True)
  (bb_layers): ModuleList(
    (0-1): 2 x ModuleList(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.01)
      (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): LeakyReLU(negative_slope=0.01)
    )
  )
  (final_convs): ModuleDict(
    (u): Conv2d(128, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
)


In [3]:
input_np = joblib.load('preData/input_data.joblib').astype(np.float32)
u_train_np = joblib.load('preData/true_output.joblib').squeeze(3).astype(np.float32)

input_train = torch.from_numpy(input_np).unsqueeze(1)
u_train = torch.from_numpy(u_train_np).unsqueeze(1)

targets_train = {"u": u_train}


In [4]:
# 定义损失函数和优化器
criterions = {
    "u": nn.MSELoss(),  # 针对 u 的损失函数
}
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [5]:
# 导入数据
train_loader, val_loader = split_data(input_train, targets_train, batch_size=1000)
trainer = Trainer(model, optimizer, criterions, device="cuda")

In [6]:
# 训练
save_path = get_next_model_filename(prefix='PIRN')
trainer.fit(train_loader, val_loader, num_epochs=100000, early_stop_patience=100, save_path=save_path)

Training:   0%|          | 0/100000 [00:00<?, ?it/s]

Epoch 1/100000 | Train Loss: 27072.9316 | Val Loss: 115333.5234
Epoch 2/100000 | Train Loss: 126797.3750 | Val Loss: 13389.4092
Epoch 3/100000 | Train Loss: 14721.1094 | Val Loss: 25504.7090
Epoch 4/100000 | Train Loss: 28041.2188 | Val Loss: 21770.2734
Epoch 5/100000 | Train Loss: 23935.8008 | Val Loss: 12383.8672
Epoch 6/100000 | Train Loss: 13616.3691 | Val Loss: 8507.4053
Epoch 7/100000 | Train Loss: 9354.6748 | Val Loss: 7103.3164
Epoch 8/100000 | Train Loss: 7810.9922 | Val Loss: 6289.0420
Epoch 9/100000 | Train Loss: 6915.7798 | Val Loss: 5512.7734
Epoch 10/100000 | Train Loss: 6062.2593 | Val Loss: 4691.6455
Epoch 11/100000 | Train Loss: 5159.4077 | Val Loss: 3817.5703
Epoch 12/100000 | Train Loss: 4198.2803 | Val Loss: 2946.3633
Epoch 13/100000 | Train Loss: 3240.3523 | Val Loss: 2183.1174
Epoch 14/100000 | Train Loss: 2401.0535 | Val Loss: 1634.7904
Epoch 15/100000 | Train Loss: 1798.0438 | Val Loss: 1313.5736
Epoch 16/100000 | Train Loss: 1444.6947 | Val Loss: 1134.3617
Epoc

KeyboardInterrupt: 

In [ ]:
trainer.plot_loss()

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
save_path = get_next_model_filename(prefix='PIRN_nonuni')
trainer.fit(train_loader, val_loader, num_epochs=100000, early_stop_patience=1000, save_path=save_path)

In [ ]:
trainer.plot_loss()